This is a simple reference for loading a geojson layer over a folium map

In [1]:
import folium
import json

In [ ]:
print(folium.__version__)

In [ ]:
%ls

### load geojson object

In [2]:
with open('Kitchener Waterloo Cambridge Neighbourhoods.geojson') as f:
    wr = json.load(f)

### create folium map

In [3]:
kw = {'location': [43.451413, -80.492713], 'zoom_start': 12}

In [4]:
m = folium.Map(**kw)

In [5]:
folium.GeoJson(wr).add_to(m)

### make sure the geojson looks correct

In [6]:
m